### IV. Scenarios

#### IV.1. Scenario A

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Data preprocessing
london_flows = pd.read_csv("london_flows.csv")

observed_flows = london_flows.pivot_table(index='station_origin', columns='station_destination', values='flows', fill_value=0).values
distance = london_flows.pivot_table(index='station_origin', columns='station_destination', values='distance', fill_value=0).values

In [3]:
station_names = np.array(london_flows['station_origin'].unique())
population = np.array([london_flows.loc[london_flows['station_origin'] == station, 'population'].iloc[0] for station in station_names], dtype=np.float64)
jobs = np.array([london_flows.loc[london_flows['station_destination'] == station, 'jobs'].iloc[0] for station in station_names], dtype=np.float64)

In [4]:
# Replace zero values with a small positive value
distance_no_zeros = np.where(distance == 0, 0.001, distance)

In [5]:
# Model calibration
beta = 2.0
epsilon = 1e-9
num_iterations = 100

In [6]:
A = np.ones(population.shape)
B = np.ones(jobs.shape)

In [7]:
for _ in range(num_iterations):
    A = 1 / (np.maximum(B * distance_no_zeros.T ** (-beta), epsilon)).sum(axis=1)
    B = 1 / (np.maximum(A * distance_no_zeros ** (-beta), epsilon)).sum(axis=0)

In [8]:
flows = np.zeros_like(observed_flows)

for i in range(A.shape[0]):
    for j in range(B.shape[0]):
        flows[i, j] = A[i] * B[j] * distance_no_zeros[i, j] ** (-beta)


In [9]:
# Scenario A
canary_wharf_index = np.where(station_names == "Canary Wharf")[0][0]
jobs_scenario_A = jobs.copy()
jobs_scenario_A[canary_wharf_index] *= 0.5

In [10]:
reduced_jobs = jobs[canary_wharf_index] * 0.5
jobs_proportions = jobs_scenario_A / jobs_scenario_A.sum()
jobs_scenario_A += reduced_jobs * jobs_proportions

首先，它创建原始作业数组的副本并将金丝雀码头的作业减少一半。然后，计算本次缩减后各站点的工作比例。reduced_jobs 变量存储金丝雀码头减少的工作数量。最后，代码根据更新的 jobs_scenario_A 数组中的比例将减少的工作重新分配给其他站点。

这种方法确保了通勤者的总数保持不变，因为从金丝雀码头移除的工作被重新分配到其他车站。

In [11]:
A_scenario_A = np.ones(population.shape)
B_scenario_A = np.ones(jobs_scenario_A.shape)

In [13]:
for _ in range(num_iterations):
    A_scenario_A = 1 / (np.maximum(B_scenario_A * distance_no_zeros.T ** (-beta), epsilon)).sum(axis=1)
    B_scenario_A = 1 / (np.maximum(A_scenario_A * distance_no_zeros ** (-beta), epsilon)).sum(axis=0)

flows_scenario_A = np.zeros_like(observed_flows)

In [14]:
for i in range(A_scenario_A.shape[0]):
    for j in range(B_scenario_A.shape[0]):
        flows_scenario_A[i, j] = A_scenario_A[i] * B_scenario_A[j] * distance_no_zeros[i, j] ** (-beta)


In [15]:
# Compare the new flows for Scenario A to the original flows
print("Original flows:\n", flows)
print("\nFlows for Scenario A:\n", flows_scenario_A)

Original flows:
 [[2.72479555e-03 2.72479548e-03 2.72479518e-03 ... 2.72479554e-03
  2.72479531e-03 3.19471562e-17]
 [2.95857978e-03 2.95857970e-03 2.95857938e-03 ... 4.14735047e-18
  2.95857952e-03 2.95857976e-03]
 [5.98802338e-03 5.98802321e-03 5.98802256e-03 ... 5.98802336e-03
  8.78308967e-18 5.98802332e-03]
 ...
 [2.75482085e-03 3.86172018e-18 2.75482047e-03 ... 2.75482084e-03
  2.75482060e-03 2.75482082e-03]
 [4.27350396e-03 4.27350383e-03 6.26827325e-18 ... 4.27350394e-03
  4.27350357e-03 4.27350391e-03]
 [3.26590709e-17 2.78551516e-03 2.78551486e-03 ... 2.78551523e-03
  2.78551498e-03 2.78551521e-03]]

Flows for Scenario A:
 [[2.72479555e-03 2.72479548e-03 2.72479518e-03 ... 2.72479554e-03
  2.72479531e-03 3.19471562e-17]
 [2.95857978e-03 2.95857970e-03 2.95857938e-03 ... 4.14735047e-18
  2.95857952e-03 2.95857976e-03]
 [5.98802338e-03 5.98802321e-03 5.98802256e-03 ... 5.98802336e-03
  8.78308967e-18 5.98802332e-03]
 ...
 [2.75482085e-03 3.86172018e-18 2.75482047e-03 ... 2.7548

In [ ]:
# Calculate the difference between the two matrices
flow_diff = flows_scenario_A - flows

In [17]:
# Calculate the average, median, and standard deviation of the differences
mean_diff = np.mean(flow_diff)
median_diff = np.median(flow_diff)
std_diff = np.std(flow_diff)

In [18]:
# Identify the most affected origin-destination pairs
max_diff_index = np.unravel_index(np.argmax(np.abs(flow_diff)), flow_diff.shape)
max_diff_value = flow_diff[max_diff_index]

In [19]:
# Print the results
print(f"Mean difference: {mean_diff}")
print(f"Median difference: {median_diff}")
print(f"Standard deviation of differences: {std_diff}")
print(f"Most affected origin-destination pair: {max_diff_index}")
print(f"Largest difference in flows: {max_diff_value}")

Mean difference: 3.628647739070816e-15
Median difference: 3.0357660829594124e-15
Standard deviation of differences: 3.941927961549982e-15
Most affected origin-destination pair: (327, 327)
Largest difference in flows: 1.3792786357491593e-13


In Scenario A, we assume a 50% reduction in jobs at Canary Wharf post-Brexit and reallocate these jobs proportionally to the remaining workplaces. We then use the calibration parameter β to calculate the new flows and ensure that the number of commuters is retained.

We observed a mean difference in flows between the original scenario and scenario A of 3.63e-15, while the median difference was 3.04e-15 and the standard deviation of the difference was 3.94e-15. These values indicate that, on average, the change in flows between the two scenarios was very small.

However, it is important to examine the most heavily affected origin-destination pairs to understand the impact on specific regions. In our analysis, the most affected origin-destination pair is (327, 327), with the largest difference in flows of 1.38e-13. This result suggests that, while the overall impact on flows is relatively small, some specific origin-destination pairs experience greater changes in commuter flows due to job losses at Canary Wharf.

In summary, the 50% reduction in jobs at Canary Wharf post-Brexit resulted in a small change in commuter flows on average, but some specific origin-destination pairs were more significantly affected.

#### IV.1. Scenario B

In [25]:
# Calculate the difference in flows
def calculate_differences(original, new):
    differences = new - original
    mean_diff = np.mean(np.abs(differences))
    median_diff = np.median(np.abs(differences))
    std_diff = np.std(np.abs(differences))
    max_diff_index = np.unravel_index(np.argmax(np.abs(differences)), differences.shape)
    max_diff = differences[max_diff_index]
    return mean_diff, median_diff, std_diff, max_diff_index, max_diff

In [26]:
# Define two new values for beta
beta1 = 1.5 * beta
beta2 = 2 * beta

In [27]:
# Compute flows for both beta1 and beta2
flows_beta1 = np.zeros_like(distance)
flows_beta2 = np.zeros_like(distance)

In [28]:
# Recompute flows for Scenario B using the new beta values
def compute_flows_scenario_B(distance, A, B, beta):
    return A.reshape(-1, 1) * B * distance ** (-beta)

flows_scenario_B_beta1 = compute_flows_scenario_B(distance_no_zeros, A, B, beta1)
flows_scenario_B_beta2 = compute_flows_scenario_B(distance_no_zeros, A, B, beta2)

In [29]:
# Calculate differences for beta1
mean_diff_beta1, median_diff_beta1, std_diff_beta1, max_diff_index_beta1, max_diff_beta1 = calculate_differences(flows, flows_scenario_B_beta1)

In [30]:
# Calculate differences for beta2
mean_diff_beta2, median_diff_beta2, std_diff_beta2, max_diff_index_beta2, max_diff_beta2 = calculate_differences(flows, flows_scenario_B_beta2)

In [31]:
# Print results
print("Original flows:\n", flows)
print("\nFlows for Scenario B (beta1):\n", flows_scenario_B_beta1)
print("\nFlows for Scenario B (beta2):\n", flows_scenario_B_beta2)

Original flows:
 [[2.72479555e-03 2.72479548e-03 2.72479518e-03 ... 2.72479554e-03
  2.72479531e-03 3.19471562e-17]
 [2.95857978e-03 2.95857970e-03 2.95857938e-03 ... 4.14735047e-18
  2.95857952e-03 2.95857976e-03]
 [5.98802338e-03 5.98802321e-03 5.98802256e-03 ... 5.98802336e-03
  8.78308967e-18 5.98802332e-03]
 ...
 [2.75482085e-03 3.86172018e-18 2.75482047e-03 ... 2.75482084e-03
  2.75482060e-03 2.75482082e-03]
 [4.27350396e-03 4.27350383e-03 6.26827325e-18 ... 4.27350394e-03
  4.27350357e-03 4.27350391e-03]
 [3.26590709e-17 2.78551516e-03 2.78551486e-03 ... 2.78551523e-03
  2.78551498e-03 2.78551521e-03]]

Flows for Scenario B (beta1):
 [[2.72479555e+00 2.72479548e+00 2.72479518e+00 ... 2.72479554e+00
  2.72479531e+00 3.45924521e-21]
 [2.95857978e+00 2.95857970e+00 2.95857938e+00 ... 1.55279622e-22
  2.95857952e+00 2.95857976e+00]
 [5.98802338e+00 5.98802321e+00 5.98802256e+00 ... 5.98802336e+00
  3.36379291e-22 5.98802332e+00]
 ...
 [2.75482085e+00 1.44585429e-22 2.75482047e+00 ..

In [32]:
print("\nScenario B (beta1) Results:")
print(f"Mean difference: {mean_diff_beta1}")
print(f"Median difference: {median_diff_beta1}")
print(f"Standard deviation of differences: {std_diff_beta1}")
print(f"Most affected origin-destination pair: {max_diff_index_beta1}")
print(f"Largest difference in flows: {max_diff_beta1}")


Scenario B (beta1) Results:
Mean difference: 2.5050864296965596
Median difference: 2.736985924081588
Standard deviation of differences: 2.392061645086703
Most affected origin-destination pair: (327, 114)
Largest difference in flows: 31.218747651762026


In [33]:
print("\nScenario B (beta2) Results:")
print(f"Mean difference: {mean_diff_beta2}")
print(f"Median difference: {median_diff_beta2}")
print(f"Standard deviation of differences: {std_diff_beta2}")
print(f"Most affected origin-destination pair: {max_diff_index_beta2}")
print(f"Largest difference in flows: {max_diff_beta2}")


Scenario B (beta2) Results:
Mean difference: 2507.591516126256
Median difference: 2739.7229100056693
Standard deviation of differences: 2394.4537067317897
Most affected origin-destination pair: (327, 114)
Largest difference in flows: 31249.966399413785


In Scenario B we consider two different values of the cost function parameter (beta) to reflect a significant increase in transport costs. We use beta1 = 1.5 * beta and beta2 = 2 * beta.

For scenario B with beta1, the mean difference between the original and new flows is 2.505, while the median difference is 2.737. The standard deviation of the difference is 2.392, the most affected origin-destination pair is (327, 114), and the maximum difference in flows is 31.219.

For Scenario B with beta2, the mean difference between the original and new flows is 2507.592 and the median difference is 2739.723. The standard deviation of the difference is 2394.454, the most affected origin-destination pair is (327, 114), and the maximum difference in flows is 31249.966.

Comparing these two scenarios, we can observe that Scenario B with beta2 has a more significant impact on the reallocation of flows than Scenario B with beta1, as the difference between the original and new flows is much larger in the case of beta2. This can be attributed to the higher value of the cost function parameter, which leads to a significant increase in transport costs and thus to a greater change in commuting patterns.

In conclusion, the results of our analysis show that Scenario B with beta2 has a greater impact on traffic reallocation than Scenario B with beta1. These findings can be used to understand the potential impact of increased transport costs on urban commuting patterns.

#### IV.1. Scenario C

The impact of Scenario A on the redistribution of flows is very small as the mean, median and maximum differences are very small (approximately 1e-15). This is due to a 50% reduction in the number of jobs at Canary Wharf, but the total number of commuters is retained through a proportional redistribution of the affected jobs.

Scenario B (beta1) has a more significant impact on the redistribution of flows than Scenario A. The mean and median differences are significantly higher than in Scenario A, with a maximum traffic difference of 31.219, suggesting a more significant change in commuting patterns to increase transport costs.

Scenario B (beta2) has the greatest impact on the redistribution of flows in all three scenarios. The mean and median differences are much higher than for Scenario A and Scenario B (beta1), with a maximum difference in flows of 31,249.966. This suggests that significant increases in transport costs (expressed as higher beta values) lead to more significant changes in commuting patterns.

In summary, Scenario B (beta2) has the greatest impact on the redistribution of flows between the three scenarios. The analysis shows that increased transport costs have a more significant impact on commuting patterns than a reduction in the number of jobs at Canary Wharf. This helps to inform policy decisions and infrastructure planning in the context of changing transport costs and employment patterns.